<a href="https://colab.research.google.com/github/khalidpark/project-SolarEnergyCompetition/blob/main/SolarEnergyCompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주제

시간대별 태양광 발전량 예측

# 데이터

site_info.csv / 총 4곳 발전소 정보

energy.csv / 4곳의 시계열 데이터 발전량 (KW) (25632 rows × 5 columns)

dangjin_fcst_data.csv / 예측 , 온도 , 습도 , 속도 , 클라우드

dangjin_obs_data.csv / 예측 , 온도 , 습도 , 속도 , 클라우드

ulsan_fcst_data.csv / 예측 , 온도 , 습도 , 속도 , 클라우드

ulsan_obs_data.csv / 예측 , 온도 , 습도 , 속도 , 클라우드

sample_submission.csv / 제출

# 상세

Forecast time : 예보 발표 시점
forecast : 예보 시간 
(ex - Forecast time:2018-03-01 11:00:00, forecast:4.0 => 2018-03-01 11:00:00에 발표한 2018-03-01 15:00:00 예보

예보 발표 시점 'forecast'시간 후 기상 예보
Temperature : 온도(℃)
Humidity : 습도(%)
WindSpeed : 풍속(m/s)
WindDirection : 풍향(º)
Cloud : 하늘상태(1-맑음, 2-구름보통, 3-구름많음, 4-흐림)

## (1) 시계열데이터의 시간에 forecast 시간을 더해줘야 해당 시간의 정확한 날씨값으로 활용가능

## (2) cloud의 수치는 위계가 없음을 유의, 인코더 유의해서 적용

# 평가산식

NMAE-10

4개의 발전소 발전량을 하나로 합하여 평가
4개 발전소 총 발전용량으로 정규화
발전용량의 10% 이상 발전된 데이터만으로 평가



---

# 데이터 입력 , 전처리

# ML 진행

# 제출

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/site_info.csv')

In [3]:
df.head()

,Id,Capacity,Address,InstallationAngle,IncidentAngle,Latitude,Longitude
0,당진수상태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
1,당진자재창고태양광,0.7,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
2,당진태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
3,울산태양광,0.5,울산광역시 남구 용잠로 623,20.0,20.0,35.477651,129.380778


개요정보, 활용도 X

In [4]:
df = pd.read_csv('/content/energy.csv')
df.head()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0


In [5]:
df_df = pd.read_csv('/content/dangjin_fcst_data.csv')
df_df.head(20)

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
5,2018-03-01 11:00:00,19.0,-5.0,60.0,4.3,344.0,1.0
6,2018-03-01 11:00:00,22.0,-2.0,55.0,2.0,345.0,1.0
7,2018-03-01 11:00:00,25.0,1.0,45.0,0.8,256.0,1.0
8,2018-03-01 11:00:00,28.0,3.0,50.0,2.3,270.0,1.0
9,2018-03-01 11:00:00,31.0,1.0,55.0,1.9,264.0,1.0


In [6]:
df_df.shape

(162208, 7)

# 데이터 확인결과

3월1일10시 + 7(forecast) 와

3월1일13시 + 4(forecast) 가 동시에 존재함을 확인

즉 중복 데이터들이 많이 존재함

시간과 forecast를 합치고 중복치를 제거해주는 작업 필수

다른 데이터도 비슷한 상황인지 추후 파악

우선 시계열 데이터 정리작업 진행

In [ ]:
df_df.dtypes

Forecast time     object
forecast         float64
Temperature      float64
Humidity         float64
WindSpeed        float64
WindDirection    float64
Cloud            float64
dtype: object

In [ ]:
#시계열데이터로 변환
df_df['Forecast time'] = pd.to_datetime(df_df['Forecast time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_df.dtypes

Forecast time    datetime64[ns]
forecast                float64
Temperature             float64
Humidity                float64
WindSpeed               float64
WindDirection           float64
Cloud                   float64
dtype: object

In [ ]:
df_df.head(10)

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
5,2018-03-01 11:00:00,19.0,-5.0,60.0,4.3,344.0,1.0
6,2018-03-01 11:00:00,22.0,-2.0,55.0,2.0,345.0,1.0
7,2018-03-01 11:00:00,25.0,1.0,45.0,0.8,256.0,1.0
8,2018-03-01 11:00:00,28.0,3.0,50.0,2.3,270.0,1.0
9,2018-03-01 11:00:00,31.0,1.0,55.0,1.9,264.0,1.0


In [ ]:
# forecast 더해주기위한 테스트

# x = df_df['forecast'][0]
# print(x)

4.0


In [ ]:
# forecast 더해주기위한 테스트

# df_df['Forecast time'][0] = df_df['Forecast time'][0] + pd.DateOffset(hours=x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# forecast 더해주기위한 테스트

# df_df['Forecast time'][0]

Timestamp('2018-03-06 19:00:00')

In [ ]:
for i in range(0,162208):
    x = df_df['forecast'][i]
    df_df['Forecast time'][i] = df_df['Forecast time'][i] + pd.DateOffset(hours=x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df_df.head(10)

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 15:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 18:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 21:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-02 00:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-02 03:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
5,2018-03-02 06:00:00,19.0,-5.0,60.0,4.3,344.0,1.0
6,2018-03-02 09:00:00,22.0,-2.0,55.0,2.0,345.0,1.0
7,2018-03-02 12:00:00,25.0,1.0,45.0,0.8,256.0,1.0
8,2018-03-02 15:00:00,28.0,3.0,50.0,2.3,270.0,1.0
9,2018-03-02 18:00:00,31.0,1.0,55.0,1.9,264.0,1.0


In [ ]:
df_df.tail(10)

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
162198,2021-03-02 21:00:00,37.0,1.0,80.0,2.4,330.0,1.0
162199,2021-03-03 00:00:00,40.0,0.0,85.0,1.2,85.0,1.0
162200,2021-03-03 03:00:00,43.0,-1.0,85.0,1.8,119.0,1.0
162201,2021-03-03 06:00:00,46.0,-2.0,80.0,2.3,133.0,1.0
162202,2021-03-03 09:00:00,49.0,1.0,65.0,3.0,150.0,1.0
162203,2021-03-03 12:00:00,52.0,7.0,40.0,3.2,187.0,1.0
162204,2021-03-03 15:00:00,55.0,8.0,40.0,4.5,217.0,1.0
162205,2021-03-03 18:00:00,58.0,5.0,55.0,2.2,210.0,1.0
162206,2021-03-03 21:00:00,61.0,1.0,80.0,1.9,164.0,1.0
162207,2021-03-04 00:00:00,64.0,1.0,80.0,2.1,152.0,1.0


In [ ]:
df_df = df_df.drop_duplicates(['Forecast time'], keep='first')

In [ ]:
df_df.shape

(8788, 7)

In [ ]:
df_do = pd.read_csv('/content/dangjin_obs_data.csv')
df_do.head(20)

,지점,지점명,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
0,129,서산,2018-03-01 00:00,3.1,3.6,340.0,96.0,NaN
1,129,서산,2018-03-01 01:00,2.8,0.7,140.0,97.0,NaN
2,129,서산,2018-03-01 02:00,2.6,3.2,320.0,95.0,NaN
3,129,서산,2018-03-01 03:00,2.0,1.9,230.0,97.0,NaN
4,129,서산,2018-03-01 04:00,2.2,2.1,180.0,97.0,NaN
5,129,서산,2018-03-01 05:00,4.1,4.4,270.0,97.0,NaN
6,129,서산,2018-03-01 06:00,3.5,7.9,320.0,93.0,NaN
7,129,서산,2018-03-01 07:00,2.2,6.4,290.0,86.0,NaN
8,129,서산,2018-03-01 08:00,1.0,7.7,320.0,82.0,NaN
9,129,서산,2018-03-01 09:00,0.3,8.9,320.0,71.0,NaN


In [ ]:
df_df.loc[0]

Forecast time    2018-03-01 15:00:00
forecast                           4
Temperature                        0
Humidity                          60
WindSpeed                        7.3
WindDirection                    309
Cloud                              2
Name: 0, dtype: object

In [ ]:
df_do.loc[15]

지점                        129
지점명                        서산
일시           2018-03-01 15:00
기온(°C)                    0.4
풍속(m/s)                     5
풍향(16방위)                  320
습도(%)                      56
전운량(10분위)                 NaN
Name: 15, dtype: object

# 데이터의 이해

같은 날짜의 같은 시간인데 결과가 다른 이유는 무엇인가?
데이터의 의미를 다시한번 확인하기

# 예보와 실제 예측치 값의 조화

예보와 실제 날씨값을 조화시키는 방법을 강구하거나

예보값은 버리고 실제 날씨값만으로 머신러닝 돌리고 추후 값을 예측

기준은 시계열. 흐름 그래프를 이해하고 예측하는것

예보가 1시간마다 진행되지 않는다는점

데이터를 그렇다고 다 잘라낼수도없고

예측데이터는 버리고 실제 데이터 (obs)만 활용하는 방향으로 가자

energy 에서 잘라서 각각의 obs에 타겟값으로 넣어서 모델을 2개 만든다

각각의 예측값들을 하나의 데이터로 합쳐서 제출한다



---

